# Detecting and Changing Gender markers in Ukrainian CVs

## Load Libraries

In [ ]:
#%pip install stanza

In [7]:
import stanza
import random 
import logging
import pymorphy3
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate

from tqdm import tqdm
tqdm.pandas()

# Load data (Ukr CVs)

In [3]:
df = pd.read_csv('../data/uk_data_samples.csv')
df.head()

,item_id,candidate_id,job_id,CV,Job Description,Job Position,lang
0,e765137d-40e0-5ae0-aaf9-a966f76f3621_1d98054f-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,1d98054f-4adf-5db0-8e2d-4dab904dfe44,"Комунікабельний, серйозний, старанний, швидко ...",**💎Нашому супергерою ми обіцяємо:**\r\n— цікав...,2D Artist,uk
1,e765137d-40e0-5ae0-aaf9-a966f76f3621_6addf108-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,6addf108-43e3-5677-b565-5117c619f084,"Комунікабельний, серйозний, старанний, швидко ...",Розробник та видавець мобільних ігор Mystery T...,2D Artist,uk
2,e765137d-40e0-5ae0-aaf9-a966f76f3621_13dbe82c-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,13dbe82c-a0fa-5538-82a6-64f8268ece38,"Комунікабельний, серйозний, старанний, швидко ...",Твої задачі:\r\n-створення та розробка дизайну...,2D Artist,uk
3,db4c55c9-e3e6-5d58-99ea-06697e4547e9_1d98054f-...,db4c55c9-e3e6-5d58-99ea-06697e4547e9,1d98054f-4adf-5db0-8e2d-4dab904dfe44,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...",**💎Нашому супергерою ми обіцяємо:**\r\n— цікав...,2D Artist,uk
4,db4c55c9-e3e6-5d58-99ea-06697e4547e9_6addf108-...,db4c55c9-e3e6-5d58-99ea-06697e4547e9,6addf108-43e3-5677-b565-5117c619f084,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...",Розробник та видавець мобільних ігор Mystery T...,2D Artist,uk


In [4]:
cvs = list(df["CV"].unique())

# [Research] based on LLM

## Initialize Chain

In [3]:
prompt = PromptTemplate(
    template="""Ви - експерт із перефразування тексту. Ваше завдання переписати текст так, щоб виглядало, що це написала людина {type_} роду. Не додавайте лишнього тексту та додаткових слів, можна змінювати тільки рід або відмінки слів.

Текст:```{text}```
Перефразований текст:""",
    input_variables=['type_', 'text'],
)

In [4]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42})

chain = prompt | llm

## Test LLM Approach

In [16]:
random.seed(10)
sample = random.choices(cvs, k=2)
sample


['Курси. Сертифікати.\r\n\r\nЛінкбілдінг. Онлайн курс Ігоря Рудника. \r\nSEO-школа Collaborator.\r\nтравень 2023 р.\r\n\r\nОснови веб розробки. HTML і CSS\r\nUdemy\r\nтравень 2023 р.\r\n\r\nSpecialist Internet Marketing  \r\nАкадемія інтернет-маркетингу WEBPromoExperts м. Київ\r\nвересень-грудень 2022 р\r\n\r\nSEO Specialist \r\n ITEA GIZ IT Education Academy м. Київ\r\n жовтень-листопад 2022 р\r\n\r\nВ процесі навчання виконувала наступні завдання: \r\n- збір семантичного ядра та його кластеризація,  опрацювання структури сайту;\r\n- внутрішня оптимізація (технічний аудит сайту, робота з метатегами, дублями сторінок, швидкістю завантаження сайту);\r\n- формування контент-плану, розробка технічного завдання для написання контенту;\r\n- написання оптимізованого контенту (стаття для інформаційного ресурсу);\r\n- зовнішня оптимізація (аналіз зовнішніх лінок конкурентів, пошук джерел розміщення посилань).\nOctober 2012 - January 2020\r\nNAAS of Ukraine. Institute of Potato - Accountant-eco

In [22]:
for i, cv in enumerate(sample):
    print(f"Sample {i+1}")
    print(f"Original:\n{cv}\n")
    print(f'Rewritten female:\n{chain.invoke({"text": cv, "type_":"жіночого"}).content}\n')
    print(f'Rewritten male: \n{chain.invoke({"text": cv, "type_": "чоловічого"}).content}\n')
    print("\n\n")

Sample 1
Original:
Курси. Сертифікати.

Лінкбілдінг. Онлайн курс Ігоря Рудника. 
SEO-школа Collaborator.
травень 2023 р.

Основи веб розробки. HTML і CSS
Udemy
травень 2023 р.

Specialist Internet Marketing  
Академія інтернет-маркетингу WEBPromoExperts м. Київ
вересень-грудень 2022 р

SEO Specialist 
 ITEA GIZ IT Education Academy м. Київ
 жовтень-листопад 2022 р

В процесі навчання виконувала наступні завдання: 
- збір семантичного ядра та його кластеризація,  опрацювання структури сайту;
- внутрішня оптимізація (технічний аудит сайту, робота з метатегами, дублями сторінок, швидкістю завантаження сайту);
- формування контент-плану, розробка технічного завдання для написання контенту;
- написання оптимізованого контенту (стаття для інформаційного ресурсу);
- зовнішня оптимізація (аналіз зовнішніх лінок конкурентів, пошук джерел розміщення посилань).
October 2012 - January 2020
NAAS of Ukraine. Institute of Potato - Accountant-economist

July 2008 - December 2009
LLC Bautech-Ukraine - 

# [Research and Current Approach] Classical NLP pipeline for changing gender in CVs

In [208]:
# ignore logs
logging.getLogger('stanza').setLevel(logging.ERROR)

def changer(morph_token, gender, token):
    # комунікабельний -> комунікабельна
    if morph_token.tag.POS == 'VERB' or morph_token.tag.POS == 'ADJF':
        if morph_token.tag.gender == 'masc' and gender == 'femn':
            token = morph_token.inflect({'femn'}).word

        elif morph_token.tag.gender == 'femn' and gender == 'masc':
            token = morph_token.inflect({'masc'}).word

        else:
            token = morph_token.word
    return token

def gender_changer(text: str, gender: str = 'masc') -> str:
    """
    Function for changing verbs from masculine to feminine and vice versa for Ukrainian language

    Args:
        text: Ukrainian language text
        gender: gender to which the verbs should be changed. Default: male

    Returns:
        text with changed verbs to masculine or feminine
    """

    nlp = stanza.Pipeline(lang='uk', processors='tokenize,mwt,pos,lemma,depparse')
    text_prep = text.replace(":", ".").replace(";", ".")
    doc = nlp(text_prep)
    morph = pymorphy3.MorphAnalyzer(lang='uk')
    changer_count = 0
    for sentence in doc.sentences:
        # find head for я with deprel to nsubj
        has_nsubj = any([word.deprel == 'nsubj' and word.upos not in ['ADJ', 'NOUN'] for word in sentence.words])
        head_ids = [] 
        for word in sentence.words:
            if word.text.lower() == 'я' and word.deprel == 'nsubj':
                head_ids.append(word.head)
            #print(word.text, word.deprel, word.upos, word.head)
        
        if not has_nsubj or head_ids != []:
            for word in sentence.words:
                token = word.text
                if token.lower() == "є":
                    break
                morph_token = morph.parse(token)[0]
                if (word.deprel == 'root' and sentence.words[word.head].upos != 'NOUN') or (word.id in head_ids) \
                    or ((word.upos =='ADJ') and (word.deprel != 'amod' or word.id != 1)):
                    token = changer(morph_token, gender, token)
                    if word.id == 1:
                        token = token.capitalize()
                    text = text[:word.start_char+changer_count] + token + text[word.end_char+changer_count:]
                    changer_count += len(token) - len(word.text)
    return text

## Test the pipeline

In [211]:
# test with masculine verb
text = 'Є фахова освіта. Закінчив курси Прожектор. Внутрішня оптимізація (технічний аудит сайту, робота з метатегами, дублями сторінок, швидкістю завантаження сайту); Комунікабельний, серйозний, старанний, швидко сприймаю новий матеріал, стресостійкий. Він працьовитий і комунікабельний. Сьогодні я пішов на роботу, я відпочивав, вона працювала та співала. Він працював. Компанія заробляла кошти. Пішли на роботу. Програмував цей диплом все життя.'
#text = "Компанія заробляла кошти"
print(gender_changer(text, 'masc'))
print(gender_changer(text, 'femn'))


text = """
Курси. Сертифікати.

Лінкбілдінг. Онлайн курс Ігоря Рудника. 
SEO-школа Collaborator.
травень 2023 р.

Основи веб розробки. HTML і CSS
Udemy
травень 2023 р.

Specialist Internet Marketing  
Академія інтернет-маркетингу WEBPromoExperts м. Київ
вересень-грудень 2022 р

SEO Specialist 
ITEA GIZ IT Education Academy м. Київ
жовтень-листопад 2022 р

В процесі навчання виконувала наступні завдання: 
- збір семантичного ядра та його кластеризація,  опрацювання структури сайту;
- внутрішня оптимізація (технічний аудит сайту, робота з метатегами, дублями сторінок, швидкістю завантаження сайту);
- формування контент-плану, розробка технічного завдання для написання контенту;
- написання оптимізованого контенту (стаття для інформаційного ресурсу);
- зовнішня оптимізація (аналіз зовнішніх лінок конкурентів, пошук джерел розміщення посилань).
October 2012 - January 2020
NAAS of Ukraine. Institute of Potato - Accountant-economist

July 2008 - December 2009
LLC Bautech-Ukraine - Accountant

February 2006 - June 2008
Private enterprise "Yulga" - Accountant
Looking for a project where I can get practical skills and grow professionally. Under the supervision of a specialist. Type of employment full-time or part-time, remote.
"""
print("\n\n")
print(gender_changer(text, 'masc'))
print()
print(gender_changer(text, 'femn'))

Є фахова освіта. Закінчив курси Прожектор. Внутрішня оптимізація (технічний аудит сайту, робота з метатегами, дублями сторінок, швидкістю завантаження сайту); Комунікабельний, серйозний, старанний, швидко сприймаю новий матеріал, стресостійкий. Він працьовитий і комунікабельний. Сьогодні я пішов на роботу, я відпочивав, вона працювала та співала. Він працював. Компанія заробляла кошти. Пішли на роботу. Програмував цей диплом все життя.
Є фахова освіта. Закінчила курси Прожектор. Внутрішня оптимізація (технічна аудит сайту, робота з метатегами, дублями сторінок, швидкістю завантаження сайту); Комунікабельна, серйозна, старанна, швидко сприймаю нова матеріал, стресостійка. Він працьовитий і комунікабельний. Сьогодні я пішла на роботу, я відпочивала, вона працювала та співала. Він працював. Компанія заробляла кошти. Пішли на роботу. Програмувала цей диплом все життя.




Курси. Сертифікати.

Лінкбілдінг. Онлайн курс Ігоря Рудника. 
Seo-школа Collaborator.
травень 2023 р.

Основи веб розро

## Run Changing on Ukrainian CVs dataset

In [212]:
df.head()

,item_id,candidate_id,job_id,CV,Job Description,Job Position,lang,CV_male_marked,CV_female_marked
0,e765137d-40e0-5ae0-aaf9-a966f76f3621_1d98054f-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,1d98054f-4adf-5db0-8e2d-4dab904dfe44,"Комунікабельний, серйозний, старанний, швидко ...",**💎Нашому супергерою ми обіцяємо:**\r\n— цікав...,2D Artist,uk,"Комунікабельний, серйозний, старанний, швидко ...","Комунікабельна, серйозна, старанна, швидко спр..."
1,e765137d-40e0-5ae0-aaf9-a966f76f3621_6addf108-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,6addf108-43e3-5677-b565-5117c619f084,"Комунікабельний, серйозний, старанний, швидко ...",Розробник та видавець мобільних ігор Mystery T...,2D Artist,uk,"Комунікабельний, серйозний, старанний, швидко ...","Комунікабельна, серйозна, старанна, швидко спр..."
2,e765137d-40e0-5ae0-aaf9-a966f76f3621_13dbe82c-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,13dbe82c-a0fa-5538-82a6-64f8268ece38,"Комунікабельний, серйозний, старанний, швидко ...",Твої задачі:\r\n-створення та розробка дизайну...,2D Artist,uk,"Комунікабельний, серйозний, старанний, швидко ...","Комунікабельна, серйозна, старанна, швидко спр..."
3,db4c55c9-e3e6-5d58-99ea-06697e4547e9_1d98054f-...,db4c55c9-e3e6-5d58-99ea-06697e4547e9,1d98054f-4adf-5db0-8e2d-4dab904dfe44,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...",**💎Нашому супергерою ми обіцяємо:**\r\n— цікав...,2D Artist,uk,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...","\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\..."
4,db4c55c9-e3e6-5d58-99ea-06697e4547e9_6addf108-...,db4c55c9-e3e6-5d58-99ea-06697e4547e9,6addf108-43e3-5677-b565-5117c619f084,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...",Розробник та видавець мобільних ігор Mystery T...,2D Artist,uk,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...","\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\..."


In [214]:
gender_changer(df["CV"].values[25], 'femn')

"Найбільше пишаюся унікальними проектами з 3D скульптур.\nПротягом довгого часу займаюся фріланс проектами з різних напрямків 3D - персонажі, принт, ювелірні вироби, декори, гейм моделі і багато іншого. Також маю досвід роботи 3д-візуалізатором-дизайнером на виробництві меблів.\nЯ готова до будь-яких пропозицій, пов'язаних з 3D графікою."

In [217]:
# create column CV_male_marked
df["CV_male_marked"] = df["CV"].progress_apply(lambda x: gender_changer(x, 'masc'))

100%|██████████| 450/450 [18:25<00:00,  2.46s/it] 


In [218]:
# create column CV_female_marked
df["CV_female_marked"] = df["CV"].progress_apply(lambda x: gender_changer(x, 'femn'))

100%|██████████| 450/450 [15:26<00:00,  2.06s/it]


In [222]:
df.head(10)

,item_id,candidate_id,job_id,CV,Job Description,Job Position,lang,CV_male_marked,CV_female_marked
0,e765137d-40e0-5ae0-aaf9-a966f76f3621_1d98054f-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,1d98054f-4adf-5db0-8e2d-4dab904dfe44,"Комунікабельний, серйозний, старанний, швидко ...",**💎Нашому супергерою ми обіцяємо:**\r\n— цікав...,2D Artist,uk,"Комунікабельний, серйозний, старанний, швидко ...","Комунікабельна, серйозна, старанна, швидко спр..."
1,e765137d-40e0-5ae0-aaf9-a966f76f3621_6addf108-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,6addf108-43e3-5677-b565-5117c619f084,"Комунікабельний, серйозний, старанний, швидко ...",Розробник та видавець мобільних ігор Mystery T...,2D Artist,uk,"Комунікабельний, серйозний, старанний, швидко ...","Комунікабельна, серйозна, старанна, швидко спр..."
2,e765137d-40e0-5ae0-aaf9-a966f76f3621_13dbe82c-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,13dbe82c-a0fa-5538-82a6-64f8268ece38,"Комунікабельний, серйозний, старанний, швидко ...",Твої задачі:\r\n-створення та розробка дизайну...,2D Artist,uk,"Комунікабельний, серйозний, старанний, швидко ...","Комунікабельна, серйозна, старанна, швидко спр..."
3,db4c55c9-e3e6-5d58-99ea-06697e4547e9_1d98054f-...,db4c55c9-e3e6-5d58-99ea-06697e4547e9,1d98054f-4adf-5db0-8e2d-4dab904dfe44,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...",**💎Нашому супергерою ми обіцяємо:**\r\n— цікав...,2D Artist,uk,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...","\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\..."
4,db4c55c9-e3e6-5d58-99ea-06697e4547e9_6addf108-...,db4c55c9-e3e6-5d58-99ea-06697e4547e9,6addf108-43e3-5677-b565-5117c619f084,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...",Розробник та видавець мобільних ігор Mystery T...,2D Artist,uk,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...","\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\..."
5,db4c55c9-e3e6-5d58-99ea-06697e4547e9_13dbe82c-...,db4c55c9-e3e6-5d58-99ea-06697e4547e9,13dbe82c-a0fa-5538-82a6-64f8268ece38,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...",Твої задачі:\r\n-створення та розробка дизайну...,2D Artist,uk,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...","\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\..."
6,5ca945c0-d854-5c3b-a15a-ba44e0325275_1d98054f-...,5ca945c0-d854-5c3b-a15a-ba44e0325275,1d98054f-4adf-5db0-8e2d-4dab904dfe44,"\n2 роки я працювала фріланс-іллюстратором, ст...",**💎Нашому супергерою ми обіцяємо:**\r\n— цікав...,2D Artist,uk,"\n2 роки я працював фріланс-іллюстратором, ств...","\n2 роки я працювала фріланс-іллюстратором, ст..."
7,5ca945c0-d854-5c3b-a15a-ba44e0325275_6addf108-...,5ca945c0-d854-5c3b-a15a-ba44e0325275,6addf108-43e3-5677-b565-5117c619f084,"\n2 роки я працювала фріланс-іллюстратором, ст...",Розробник та видавець мобільних ігор Mystery T...,2D Artist,uk,"\n2 роки я працював фріланс-іллюстратором, ств...","\n2 роки я працювала фріланс-іллюстратором, ст..."
8,5ca945c0-d854-5c3b-a15a-ba44e0325275_13dbe82c-...,5ca945c0-d854-5c3b-a15a-ba44e0325275,13dbe82c-a0fa-5538-82a6-64f8268ece38,"\n2 роки я працювала фріланс-іллюстратором, ст...",Твої задачі:\r\n-створення та розробка дизайну...,2D Artist,uk,"\n2 роки я працював фріланс-іллюстратором, ств...","\n2 роки я працювала фріланс-іллюстратором, ст..."
9,b6ff12af-4bac-54fe-b2ac-f9a98d7dd50c_bb1895c9-...,b6ff12af-4bac-54fe-b2ac-f9a98d7dd50c,bb1895c9-bb30-5659-bd6d-7cc2c73623eb,Є фахова освіта: Харківський професійний худож...,Привіт! 🙂 \r\n\r\nМи шукаємо спеціаліста на п...,2D Artist,uk,Є фахова освіта: Харківський професійний худож...,Є фахова освіта: Харківський професійна художн...


In [223]:
df.tail(10)

,item_id,candidate_id,job_id,CV,Job Description,Job Position,lang,CV_male_marked,CV_female_marked
440,da1129a0-76e9-5417-9eaf-808bacd7821e_5159b5da-...,da1129a0-76e9-5417-9eaf-808bacd7821e,5159b5da-44b0-5dc1-8f98-1b4d7ea6dee8,\nВиконані проекти: по створенню лейдинга для ...,"Робота тільки офіс! — Київ, Україна🇺🇦\r\n\r\nС...",Дизайнер,uk,\nВиконані проекти: по створенню лейдинга для ...,\nВиконані проекти: по створенню лейдинга для ...
441,e1eec496-eff7-5fff-b052-02c97746fa59_c0bc2a3e-...,e1eec496-eff7-5fff-b052-02c97746fa59,c0bc2a3e-8960-51a7-bf19-064356b68c62,"\nВища освіта редактор-видавець (2018р), звідс...","Для нашого клієнта, ІТ - холдингу, шукаємо Диз...",Дизайнер,uk,"\nВища освіта редактор-видавець (2018р), звідс...","\nВища освіта редактор-видавець (2018р), звідс..."
442,e1eec496-eff7-5fff-b052-02c97746fa59_c5a40093-...,e1eec496-eff7-5fff-b052-02c97746fa59,c5a40093-30c4-51e3-aec3-74efeb7ce5aa,"\nВища освіта редактор-видавець (2018р), звідс...",Опис вакансії\r\nМи IAMPM 💙 — лабораторія онла...,Дизайнер,uk,"\nВища освіта редактор-видавець (2018р), звідс...","\nВища освіта редактор-видавець (2018р), звідс..."
443,e1eec496-eff7-5fff-b052-02c97746fa59_5159b5da-...,e1eec496-eff7-5fff-b052-02c97746fa59,5159b5da-44b0-5dc1-8f98-1b4d7ea6dee8,"\nВища освіта редактор-видавець (2018р), звідс...","Робота тільки офіс! — Київ, Україна🇺🇦\r\n\r\nС...",Дизайнер,uk,"\nВища освіта редактор-видавець (2018р), звідс...","\nВища освіта редактор-видавець (2018р), звідс..."
444,52dd1544-f02a-541d-8ddd-2f94aa800a31_c0bc2a3e-...,52dd1544-f02a-541d-8ddd-2f94aa800a31,c0bc2a3e-8960-51a7-bf19-064356b68c62,Швидко опанував основані функції програми для ...,"Для нашого клієнта, ІТ - холдингу, шукаємо Диз...",Дизайнер,uk,Швидко опанував основані функції програми для ...,Швидко опанувала основані функції програми для...
445,52dd1544-f02a-541d-8ddd-2f94aa800a31_c5a40093-...,52dd1544-f02a-541d-8ddd-2f94aa800a31,c5a40093-30c4-51e3-aec3-74efeb7ce5aa,Швидко опанував основані функції програми для ...,Опис вакансії\r\nМи IAMPM 💙 — лабораторія онла...,Дизайнер,uk,Швидко опанував основані функції програми для ...,Швидко опанувала основані функції програми для...
446,52dd1544-f02a-541d-8ddd-2f94aa800a31_5159b5da-...,52dd1544-f02a-541d-8ddd-2f94aa800a31,5159b5da-44b0-5dc1-8f98-1b4d7ea6dee8,Швидко опанував основані функції програми для ...,"Робота тільки офіс! — Київ, Україна🇺🇦\r\n\r\nС...",Дизайнер,uk,Швидко опанував основані функції програми для ...,Швидко опанувала основані функції програми для...
447,87f71a5b-2017-5910-ab93-30f04453cdd3_c0bc2a3e-...,87f71a5b-2017-5910-ab93-30f04453cdd3,c0bc2a3e-8960-51a7-bf19-064356b68c62,Закінчив курси Прожектор beginner та junior))\...,"Для нашого клієнта, ІТ - холдингу, шукаємо Диз...",Дизайнер,uk,Закінчив курси Прожектор beginner та junior))\...,Закінчила курси Прожектор beginner та junior))...
448,87f71a5b-2017-5910-ab93-30f04453cdd3_c5a40093-...,87f71a5b-2017-5910-ab93-30f04453cdd3,c5a40093-30c4-51e3-aec3-74efeb7ce5aa,Закінчив курси Прожектор beginner та junior))\...,Опис вакансії\r\nМи IAMPM 💙 — лабораторія онла...,Дизайнер,uk,Закінчив курси Прожектор beginner та junior))\...,Закінчила курси Прожектор beginner та junior))...
449,87f71a5b-2017-5910-ab93-30f04453cdd3_5159b5da-...,87f71a5b-2017-5910-ab93-30f04453cdd3,5159b5da-44b0-5dc1-8f98-1b4d7ea6dee8,Закінчив курси Прожектор beginner та junior))\...,"Робота тільки офіс! — Київ, Україна🇺🇦\r\n\r\nС...",Дизайнер,uk,Закінчив курси Прожектор beginner та junior))\...,Закінчила курси Прожектор beginner та junior))...


In [224]:
df.to_csv('../data/uk_data_samples.csv', index=False)